<a href="https://colab.research.google.com/github/fancagi/intro_pulp/blob/main/Localizacio%CC%81n_de_instalaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema de localización de instalaciones

## Objetivo y Prerrequisitos
En este ejemplo, resolveremos un problema de ubicación de instalaciones donde queremos construir almacenes para abastecer a un cierto número de supermercados. Construiremos un modelo de programación entera mixta (MIP) de este problema, implementaremos este modelo en la interfaz de Python de Gurobi y calcularemos una solución óptima.

Este ejemplo de modelado está dirigido a nivel principiante, donde se asume que usted conoce Python y tiene algún conocimiento sobre la construcción de modelos de optimización matemática.

## Motivación

El estudio de los problemas de localización de instalaciones, también conocidos como análisis de ubicación [1], es una rama de la investigación de operaciones y la geometría computacional que se preocupa por la ubicación óptima de instalaciones para minimizar los costos ***e.g.*** de transporte, mientras se consideran factores como evitar colocar materiales peligrosos cerca de viviendas y la ubicación de las instalaciones de competidores.

El problema de Fermat-Weber, formulado en el siglo XVII, fue uno de los primeros problemas de ubicación de instalaciones que se propusieron. El problema de Fermat-Weber se puede describir de la siguiente manera: 

>dados tres puntos en un plano, encontrar un cuarto punto tal que la suma de sus distancias a los tres puntos dados sea mínima. Este problema se puede interpretar como una versión del problema de ubicación de instalaciones, donde se hace la suposición de que los costos de transporte por distancia son iguales para todos los destinos.

Los problemas de ubicación de instalaciones tienen aplicaciones en una amplia variedad de industrias. Para la gestión de la cadena de suministro y la logística, este problema se puede utilizar para encontrar la ubicación óptima de tiendas, fábricas, almacenes, etc. Otras aplicaciones van desde la política pública (por ejemplo, la ubicación de policías en una ciudad), las telecomunicaciones (por ejemplo, torres de celulares en una red) e incluso la física de partículas (por ejemplo, la distancia de separación entre cargas repulsivas). Otra aplicación del problema de ubicación de instalaciones es determinar las ubicaciones para el equipo de transmisión de gas natural. Finalmente, los problemas de ubicación de instalaciones se pueden aplicar al análisis de conglomerados.

## Descripción del problema

Una red de hospitales en Colombia necesita construir almacenes para suministrar insumos médicos a sus sedes en el norte del país. Las ubicaciones de los hospitales ya han sido decididas, pero aún no se ha determinado la ubicación de los almacenes.

Se han identificado varias ubicaciones potenciales para los almacenes, pero se deben tomar decisiones sobre cuántos almacenes abrir y en qué ubicaciones candidatas construirlos.

Abrir muchos almacenes sería ventajoso, ya que esto reduciría la distancia promedio que un camión tendría que recorrer desde el almacén hasta el hospital, y por lo tanto, reduciría el costo de entrega. Sin embargo, la apertura de un almacén tiene un costo fijo asociado.

En este ejemplo, nuestro objetivo es encontrar el equilibrio óptimo entre el costo de entrega y el costo de construir nuevas instalaciones.

## Solución planteada

La programación matemática es un enfoque declarativo en el que el modelador formula un modelo de optimización matemática que captura los aspectos clave de un problema de decisión complejo. El optimizador Gurobi resuelve tales modelos utilizando matemáticas y ciencias de la computación de última generación.

Un modelo de optimización matemática tiene cinco componentes, a saber:

* Conjuntos e índices.
* Parámetros.
* Variables de decisión.
* Función(es) objetivo.
* Restricciones.

Presentamos a continuación una formulación MIP para el problema de ubicación de instalaciones.


## Formulación del Modelo

### Conjuntos e Índices
$i \in I$: Índice y conjunto de ubicaciones de hospitales (o clientes).

$j \in J$: Índice y conjunto de ubicaciones candidatas de almacenes (o instalaciones).

### Parámetros
$f_{j} \in \mathbb{R}^+$: Costo fijo asociado con la construcción de la instalación $j \in J$.

$d_{i,j} \in \mathbb{R}^+$: Distancia entre la instalación $j \in J$ y el cliente $i \in I$.

$c_{i,j} \in \mathbb{R}^+$: Costo de envío entre el sitio candidato de la instalación $j \in J$ y la ubicación del cliente $i \in I$. Se asume que este costo es proporcional a la distancia entre la instalación y el cliente. Es decir, $c_{i,j} = \alpha \cdot d_{i,j}$, donde $\alpha$ es el costo por milla de conducción, ajustado para incorporar el número promedio de viajes que se esperaría que un camión de entrega realizara durante un período de cinco años.

### Variables de Decisión
$select_{j} \in {0, 1 }$: Esta variable es igual a 1 si construimos una instalación en la ubicación candidata $j \in J$; y 0 en caso contrario.

$0 \leq assign_{i,j} \leq 1$: Esta variable continua no negativa determina la fracción de suministro recibida por el cliente $i \in I$ de la instalación $j \in J$.

### Función Objetivo
Costos totales. Queremos minimizar el costo total de abrir y operar las instalaciones. Esta es la suma del costo de abrir las instalaciones y el costo relacionado con el envío entre las instalaciones y los clientes. Este costo total mide el compromiso entre el costo de construir una nueva instalación y el costo total de envío durante un período de cinco años.
\begin{equation}
\text{Max} \quad Z = \sum_{j \in J} f_{j} \cdot select_{j} + \sum_{j \in J} \sum_{i \in I} c_{i,j} \cdot assign_{i,j}
\tag{0}
\end{equation}

### Restricciones
Demanda. Para cada cliente $i \in I$, aseguramos que se cumpla su demanda. Es decir, la suma de la fracción recibida de cada instalación para cada cliente debe ser igual a 1:
\begin{equation}
\sum_{j \in J} assign_{i,j} = 1 \quad \forall i \in I
\tag{1}
\end{equation}

Envío. Debemos asegurarnos de que solo enviemos desde la instalación $j \in J$ si esa instalación realmente ha sido construida.
\begin{equation}
assign_{i,j} \leq select_{j} \quad \forall i \in I \quad \forall j \in J
\tag{2}
\end{equation}

## Python Implementation

This example considers two supermarkets and nine warehouse candidates. The coordinates of each supermarket are provided in the following table.

| <i></i> | Coordinates |  
| --- | --- | 
| Supermarket 1 | (0,1.5) |
| Supermarket 2 | (2.5,1.2) |

The following table shows the coordinates of the candidate warehouse sites and the fixed cost of building the warehouse in millions of GBP.

| <i></i> | coordinates | fixed cost |
| --- | --- |  --- |
| Warehouse 1 | (0,0) | 3 |
| Warehouse 2 | (0,1) | 2 |
| Warehouse 3 | (0,2) | 3 |
| Warehouse 4 | (1,0) | 1 |
| Warehouse 5 | (1,1) | 3 | 
| Warehouse 6 | (1,2) | 3 |
| Warehouse 7 | (2,0) | 4 |
| Warehouse 8 | (2,1) | 3 |  
| Warehouse 9 | (2,2) | 2 |


The cost per mile is $\$1$ million GBP.

## Python Implementation

We now import the Gurobi Python Module and other Python libraries. Then, we initialize the data structures with the given data.

In [ ]:
from itertools import product
from math import sqrt

import pulp as plp


# tested with Gurobi v9.0.0 and Python 3.7.0

# Parameters
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Plot the problem configuration (facilities and customers)
fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x,y in facilities], y=[y for x,y in facilities], mode='markers', name='Facilities'))
fig.add_trace(go.Scatter(x=[x for x,y in customers], y=[y for x,y in customers], mode='markers', name='Customers'))
fig.update_layout(title='Facilities and customers', xaxis_title='x', yaxis_title='y')
fig.show()

### Preprocesamiento
Definimos una función que determina la distancia euclidiana entre cada instalación y los sitios de los clientes. Además, calculamos los parámetros clave requeridos por la formulación del modelo MIP del problema de ubicación de la instalación.

In [ ]:
# This function determines the Euclidean distance between a facility and customer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

# Compute key parameters of MIP model formulation

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# Compute shipping costs

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

## Despliegue del modelo

Ahora definimos el modelo MIP para el problema de ubicación de instalaciones, mediante la definición de las variables de decisión, restricciones y función objetivo. A continuación, iniciamos el proceso de optimización y Gurobi encuentra el plan para construir instalaciones que minimiza los costos totales.

In [ ]:
import pulp as pl
from itertools import product

# MIP model formulation
m = pl.LpProblem('facility_location', pl.LpMinimize)

select = pl.LpVariable.dicts('Select', range(num_facilities), lowBound=0, upBound=1, cat='Binary')
assign = pl.LpVariable.dicts('Assign', cartesian_prod, lowBound=0, upBound=1, cat='Continuous')

m += pl.lpSum([select[f]*setup_cost[f] for f in range(num_facilities)]) + \
     pl.lpSum([assign[(c,f)]*shipping_cost[(c,f)] for (c,f) in product(range(num_customers), range(num_facilities))])
               

for c,f in cartesian_prod:
     m += assign[(c,f)] <= select[f]
      
for c in range(num_customers):     
     m += pl.lpSum([assign[(c,f)] for f in range(num_facilities)]) == 1 

m.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/opt/anaconda3/envs/apricot-env/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/63427875e3c04b79ba54247c12e108fd-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/63427875e3c04b79ba54247c12e108fd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 125 RHS
At line 146 BOUNDS
At line 174 ENDATA
Problem MODEL has 20 rows, 27 columns and 54 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4.72371 - 0.00 seconds
Cgl0004I processed model has 20 rows, 27 columns (9 integer (9 of which binary)) and 54 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 4.72371
Cbc0038I Relaxing continuous gives 4.72371
Cbc0038I Befor

1

e mini branch and bound, 9 integers at bound fixed and 18 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of 4.72371 - took 0.00 seconds
Cbc0012I Integer solution of 4.7237129 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 4.723712908962, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 4.72371 to 4.72371
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds o

## Análisis
El resultado del modelo de optimización muestra que el valor mínimo del costo total es de 4.72 millones de GBP. Veamos la solución que logra ese resultado óptimo.

## Plan de construcción de almacenes
Este plan determina en qué ubicaciones de sitios construir un almacén.

In [ ]:
# display optimal values of decision variables using pulp notation

for k in select:
    print(select[k].name, '=', select[k].varValue)

for k in assign:
    print(assign[k].name, '=', assign[k].varValue)

Select_0 = 0.0
Select_1 = 0.0
Select_2 = 0.0
Select_3 = 1.0
Select_4 = 0.0
Select_5 = 0.0
Select_6 = 0.0
Select_7 = 0.0
Select_8 = 0.0
Assign_(0,_0) = 0.0
Assign_(0,_1) = 0.0
Assign_(0,_2) = 0.0
Assign_(0,_3) = 1.0
Assign_(0,_4) = 0.0
Assign_(0,_5) = 0.0
Assign_(0,_6) = 0.0
Assign_(0,_7) = 0.0
Assign_(0,_8) = 0.0
Assign_(1,_0) = 0.0
Assign_(1,_1) = 0.0
Assign_(1,_2) = 0.0
Assign_(1,_3) = 1.0
Assign_(1,_4) = 0.0
Assign_(1,_5) = 0.0
Assign_(1,_6) = 0.0
Assign_(1,_7) = 0.0
Assign_(1,_8) = 0.0


In [ ]:
# Depict the optimal solution in plotly figure, use lines to depict  allocation of customers to facilities

fig = go.Figure()
fig.add_trace(go.Scatter(x=[x for x,y in facilities], y=[y for x,y in facilities], mode='markers', name='Facilities'))
fig.add_trace(go.Scatter(x=[x for x,y in customers], y=[y for x,y in customers], mode='markers', name='Customers'))
for c,f in cartesian_prod:
    if assign[(c,f)].varValue > 0:
        fig.add_trace(go.Scatter(x=[customers[c][0], facilities[f][0]], y=[customers[c][1], facilities[f][1]], mode='lines', line=dict(color='gray', width=1), showlegend=False))
fig.update_layout(title='Facilities and customers', xaxis_title='x', yaxis_title='y')
fig.show()

### Plan de Envío
Este plan determina el porcentaje de envíos que se enviarán desde cada instalación construida a cada cliente.

In [ ]:
# Shipments from facilities to customers.

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].varValue) > 1e-6):
        print(f"\n Hospital {customer + 1} receives from Warehouse {facility + 1} {round(100*assign[customer, facility].varValue)}% of its needs")



 Hospital 1 receives from Warehouse 4 100% of its needs

 Hospital 2 receives from Warehouse 4 100% of its needs


##  Conclusión
En este ejemplo, abordamos un problema de ubicación de instalaciones en el que deseamos construir almacenes para suministrar a una gran cantidad de hospitales, minimizando los costos totales fijos de construcción de almacenes y los costos variables totales de envío desde los almacenes hasta los hospitales. Aprendimos cómo formular el problema como un modelo MIP. También aprendimos cómo implementar la formulación del modelo MIP y resolverlo utilizando la API de Python de Cbc Solver. 

##  Referencias
[1] Laporte, Gilbert, Stefan Nickel, and Saldanha da Gama, Francisco. Location Science. Springer, 2015.

Copyright © 2020 Gurobi Optimization, LLC